# Regresión Lineal Simple. Ejemplo minimalista

### Importar las librerías relevantes

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Para graficar en 3-D

### Generar datos al azar para entrenar al modelo

Trabajaremos con dos variables de entrada, las x1 y x2 en nuestros ejemplos anteriores. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [2]:
# Por facilidad, declaramos una variable que indique el tamaño del conjunto 
#      de datos de entrenamiento.
observaciones = 1000

x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

X = np.column_stack((x1,x2))

# Verificar la forma de la matriz 
# Debiera ser n x k, donde n es el número de observaciones, y k es el número de variables.
print (X.shape)

(1000, 2)


### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido.  

In [3]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

metas = 2 * x1 - 3 * x2 + 5 + ruido

# Veamos las dimensiones. Deben ser n x m, donde m es el número de variables de salida.
print (metas.shape)

(1000, 1)


### Graficar los datos a usar para el entrenamiento

La idea es ver que haya una fuerte tendencia que nuestro modelo debe aprender a reproducir.


In [4]:
print(x1.shape)
print(x2.shape)
print(metas.shape)

(1000, 1)
(1000, 1)
(1000, 1)


In [5]:
x1N = x1.reshape(observaciones,)
x2N = x2.reshape(observaciones,)
metasN = metas.reshape(observaciones,)

fig = px.scatter_3d(x = x1N, y = x2N, z = metasN)

fig.update_layout(
    width = 500,
    height = 500,)

fig.show()

### Inicializar variables

Inicializaremos los pesos y sesgos, al azar, dentro de un rango inicial pequeño.  Es posible "jugar" con este valor pero no es recomendable ya que el uso de rangos iniciales altos inhibe el aprendizaje por parte del algoritmo

Los pesos son de dimensiones k x m, donde k es el numero de variables de entrada y m es el número de variables de salida.  

Como solo hay una salida, el sesgo es de tamaño 1, y es un escalar

In [6]:
rango_inicial = 0.1

pesos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=(2, 1))

sesgos = np.random.uniform(low = -rango_inicial, high = rango_inicial, size=1)

#Veamos cómo fueron inicializados.
print (pesos)
print (sesgos)

[[ 0.07591905]
 [-0.09373346]]
[0.05751666]


In [7]:
pesos.shape

(2, 1)

### Asignar la tasa de aprendizaje (Eta)

Se asigna un a tasa de aprendizaje pequeña.  Para este ejemplo funciona bien 0.02.  Vale la pena "jugar" con este valor para ver los resultados de hacerlo.

In [8]:
eta = 0.01

### Entrenar el modelo

Usaremos un valor de 100 para iterar sobre el conjunto de datos de entrenamiento.  Ese valor funciona bastante bien con la tasa de aprendizaje de 0.02.  Cómo saber el número adecuado de iteraciones es algo que veremos en futuras sesiones, pero generalmente una tasa de aprendizaje baja requiere de más iteraciones que una más alta.  Sin embargo hay que tener en mente que una tasa de aprendizaje alta puede causar que la pérdida "Loss" diverja a infinito, en vez de converger a cero (0)

Usaremos la función de pérdida L2-norm, pero dividido por 2, para ser consistente con la clase.  Es más, también lo dividiremos por el número de observaciones para obtener un promedio de pérdida por observación.  Hablamos en clase sobre la posiilidad de modificar esta función una vez no se pierda la característica de ser más baja para los resultados mejores, y vice versa.

Imprimimos la función de pérdida (loss) en cada iteración, para ver si está decreciendo como se desea.

Otro pequeño truco es escalar las deltas de la misma manera que se hizo con la función de pérdida.  De esta forma la tasa de aprendizaje es independiente del número de muestras (samples u observaciones).  De nuevo esto no cambia el principio, solo hace más fácil la selección de una tasa única de aprendizaje. 

Finalmente aplicamos la regla de actualización del decenso de gradiente.

Ojo!  los pesos son 2x1, la tasa de aprendizaje es 1x1 (escalar), las entradas son 1000x2, y las deltas escaladas son 1000x1.  Necesitamos obtener la transpuesta de las entradas para que no hayan problemas de dimensión en las operaciones. 



In [9]:
for i in range (100):
    
    # Esta es la ecuacion del modelo lineal: y = xw + b 
    y = np.dot(X, pesos) + sesgos
    
    # Las deltas son las diferencias entre las salidas y las metas (targets)
    # deltas es un vector 1000 x 1
    deltas = y - metas
        
    perdida = np.sum(deltas ** 2) / 2 / observaciones
    
    print(perdida)
    
    deltas_escaladas = deltas / observaciones
      
    pesos = pesos - eta * np.dot(X.T, deltas_escaladas)
    sesgos = sesgos - eta * np.sum(deltas_escaladas)
    
    # Los pesos son actualizados en una forma de algebra lineal(una matriz menos otra)
    # Sin embargo, los sesgos en este caso son solo un número (solo estamos calculando una salida), 
    #       es necesario transformar las deltas a un escalar.      
    # Ambas lineas son consistentes con la metodología de decenso de gradiente-

216.01812481554785
100.97218190220947
50.802526557075694
28.80564807633544
19.068891475096954
14.680116699114919
12.630720165681927
11.60835322883334
11.039270857346056
10.67203033200015
10.396146974433263
10.162928059677963
9.950834620680052
9.750290291818331
9.557002770324095
9.369020350810807
9.18543130200023
9.005787652187522
8.829849224040528
8.657469617652133
8.488545278337865
8.322992675748564
8.1607380434568
8.001712745866742
7.845851173102983
7.693089774670098
7.543366607554882
7.396621117456526
7.25279402606143
7.111827266807275
6.973663943004171
6.838248296417608
6.705525680883597
6.575442538469949
6.447946377040401
6.322985748692451
6.200510228821103
6.080470395689957
5.962817810450682
5.847504997579324
5.734485425710702
5.623713488858162
5.515144488008753
5.408734613085338
5.304440925267837
5.20222133966625
5.102034608338365
5.003840303645226
4.907598801937638
4.81327126756711
4.720819637214772
4.630206604531971
4.541395605086273
4.454350801606914
4.369037069523613
4.28541

### Desplegamos los pesos y el sesgo para ver si funcionaron correctamente.

Por el diseño de nuestro datos, los pesos debieran ser 2 y -3, y el sesgo: 5

**NOTA:**  Si aún no están los valores correctos, puede que aún estén convergiendo y sea necesario iterar más veces.  Para esto solo se requiere ejecutar la celda anterior cuantas veces sea requerido

In [10]:
print(pesos, sesgos)      

[[ 1.98964723]
 [-2.99495956]] [3.20465763]


### Graficar las últimas salidas vrs las metas (targets)

Como son las últimas, luego del entrenamiento, representan el modelo final de exactitud.  Entre más cercana esté esta gráfica a una línea de 45 grados, más cercanas están las salidas y metas.

Como este ejemplo es pequeño, es posible hacerlo, en los problemas que veremos más tarde en la clase, esto ya no sería posible.

In [11]:
yN = y.reshape(observaciones,)
metasN = metas.reshape(observaciones,)
fig = px.scatter(x = yN, y =  metasN)

fig.update_layout(
    width = 400,
    height = 400,)

fig.show()